# Bokeh

## ColumnDataSource

接受三种数据传入形式，分别为：

1. Dict<p>
data = {'x': [1,2,3,4], 'y': np.ndarray([10.0, 20.0, 30.0, 40.0])}<p>
source = ColumnDataSource(data)<p>
2. DataFrame Objec<p>
source = ColumnDataSource(df)<p>
3. GroupBy Object<p>
group = df.groupby(('colA', 'ColB'))<p>
source = ColumnDataSource(group)    
接着使用ColumnDataSource对返回数据进行处理生成source，在绘制图形的过程中，将source作为参数传入，此时通过列名便能获取到对应的数据。

## Bokeh中绘图的一般步骤
* 加载 bokeh 库，声明在 notebook 或 html 文件中显示或输出绘制的图表

* 绘制图表框架 figure()

* 在 figure 上绘制具体的图形，比如 circle，line，bar等

* 显示图片，show()

In [ ]:
# 加载
import pandas as pd
import numpy as np
# 导入notebook绘图模块

from bokeh.plotting import figure
from bokeh.io import show, output_notebook
from bokeh.models import CustomJS, ColumnDataSource,WidgetBox,HoverTool,Button, RadioButtonGroup, Select, Slider,CheckboxGroup, CustomJSFilter, CDSView, IndexFilter
from bokeh.core.properties import value
from bokeh.models.annotations import Span
from bokeh.layouts import column,gridplot,row

output_notebook()

In [ ]:
## Import data source'
dict_source={'LB':'lb.csv'}
for k,v in dict_source.items():
    globals()['df_'+k]=pd.read_csv('./data/bokeh/'+v)  

In [ ]:
# make_dataset
lst_AnalyteName=['ALT','AST','BILI']
dict_color={'ALT':'red','AST':'olive','BILI':'blue'}
df_source= df_LB[df_LB['Subject']=='UAT012015']

In [ ]:
# figure
p = figure(plot_width=800, plot_height=300,toolbar_location="above",
           x_range=(0,50),
           y_range=(0,60),
           title="Patient XXXXX"
           )  

In [ ]:
# figure style
p.xaxis.axis_label = "days_from_baseline"
p.yaxis.axis_label = "NumericValue" 

In [ ]:
p.line(x=df_source[df_source['AnalyteName']=='ALT']['days_from_baseline'],
       y=df_source[df_source['AnalyteName']=='ALT']['NumericValue'],
       line_width=2,
       color=dict_color['ALT'])

In [ ]:
for test in lst_AnalyteName:
    p.line(df_source[df_source['AnalyteName']==test]['days_from_baseline'],
           df_source[df_source['AnalyteName']==test]['NumericValue'],
           x='days_from_baseline',
           y='NumericValue',
           line_width=2,
           color=dict_color[test], 
           legend_label = test
#             alpha=0.8,
           )

In [ ]:
p.legend.location = "top_left"
p.legend.click_policy="hide"

In [ ]:
show(p)

## bokeh的交互·主要有两种类型
一种是被动式（passive）的Inspector，一种是主动式（active）的Widgets。以下简单列举本次构建的APP涉及到的交互工具，并做简单说明介绍：

### 被动式（passive）的Inspector
HoverTool: 默认情况下，悬浮工具将生成“表格”类工具提示，其中每行包含标签及其关联信息。每个标签和对应信息构成一个元组，所有标签和信息构建成一个元组的列表作为参数进行传递。

检查器是基于当前光标位置对图像进行注释或报告图像信息的被动工具。

In [ ]:
def UnstackToFatDf(dfn,lst_key,lst_var,lv):
    df_temp=dfn.copy()
    df_temp=df_temp[lst_key+lst_var]
    df_temp=df_temp.set_index(lst_key)
    df_temp_unstack=df_temp.unstack(level=lv)
    df_temp_unstack.columns=df_temp_unstack.columns.droplevel()
    df_temp_unstack=df_temp_unstack.reset_index()
    return df_temp_unstack

In [ ]:
df_eDISH=df_LB[df_LB['AnalyteName'].isin(['AST','ALT','BILI'])]
df_eDISH['VALUE/ULN']=df_eDISH['NumericValue']/df_eDISH['LabHigh']
df_eDISH_Peak=df_eDISH.groupby(['Subject','AnalyteName'])['VALUE/ULN'].max().reset_index()
df_eDISH_Peak_Unstack=UnstackToFatDf(df_eDISH_Peak,['Subject','AnalyteName'],['VALUE/ULN'],1)
df_eDISH_Peak_Unstack=df_eDISH_Peak_Unstack.rename(columns={'AST':'Peak_AST','ALT':'Peak_ALT','BILI':'Peak_BILI'})
# df_eDISH_Peak_Unstack.fillna(0,inplace=True)
# df_eDISH_Peak_Unstack

In [ ]:
# df_eDISH_Peak_Unstack.index.name = 'index'
source = ColumnDataSource(df_eDISH_Peak_Unstack)

In [ ]:
hover = HoverTool(tooltips=[("Peak_AST", "@Peak_AST"),
                            ("Peak_BILI", "@Peak_BILI"),
                            ("Subject","@Subject")
                        ])

In [ ]:
p1 = figure(plot_width=800, plot_height=300,toolbar_location="above",
            x_range=(0,2.5),
            y_range=(0,4.5),
            tools=[hover,'box_select,reset,wheel_zoom,pan,crosshair'],
            title="eDISH"
           )   # 注意这里书写方式;  hover它的作用是只是会显示出点的每个标签;crossshair是显示十字叉

In [ ]:
# 如果不设置标签，就只写hover，例如 tools='hover,box_select,reset,wheel_zoom,pan,crosshair'
p1.circle(x = 'Peak_AST',
          y = 'Peak_BILI',
          source = source,
          size = 7,
#           alpha = 0.3        
         )

In [ ]:
p1.background_fill_color = "beige"    # 绘图空间背景颜色
p1.background_fill_alpha = 0.5        # 绘图空间背景透明度

p1.border_fill_color = "whitesmoke"    # 外边界背景颜色
p1.border_fill_alpha = 0.5             #透明度
p1.min_border_left = 80                # 外边界背景 - 左边宽度
p1.min_border_right = 80               # 外边界背景 - 右边宽度
p1.min_border_top = 10                 # 外边界背景 - 上宽度
p1.min_border_bottom = 10 

p1.xaxis.axis_label = "Peak_AST"
p1.xaxis.axis_line_width = 0.5
# p1.xaxis.bounds = (0, 3)

p1.yaxis.axis_label = "Peak_BILI"
# p1.yaxis.major_label_text_color = "orange"
# p1.yaxis.major_label_orientation = "vertical"

# p1.axis.minor_tick_in = 20     # 刻度往绘图区域内延伸长度；设置成负的就是往外边延伸了。
# p1.axis.minor_tick_out = 3   # 刻度往绘图区域外延伸长度

In [ ]:
upper1 = Span(location=3,           # 设置位置，对应坐标值
             dimension='width',    # 设置方向，width为横向，height为纵向  
             line_color='olive', line_width=2,   # 设置线颜色、线宽
             line_dash = [8,4]
            )
upper2 = Span(location=2,dimension='height',line_color='firebrick', line_width=2,line_dash = [8,4])
p1.add_layout(upper1)
p1.add_layout(upper2)

In [ ]:
show(p1)

### 主动式（active）的Widgets


窗口小部件是可以添加到bokeh应用程序的交互式控件，以便为可视化提供前端用户界面。窗口小部件可以驱动程序进行新的运算，更新绘图，甚至连接到其他程序。

CheckBoxGroup: 标准的复选框，支持多个指标同时选定

DatePicker: 更新日期

Button: 按钮，点击触发运算

Select: 单个选择工具，选项之间互斥

In [ ]:
# create some widgets
slider = Slider(start=0, end=10, value=1, step=.1, title="Slider")
button_group = RadioButtonGroup(labels=["Option 1", "Option 2", "Option 3"], active=0)
select = Select(title="Option:", value="foo", options=["foo", "bar", "baz", "quux"])
button_1 = Button(label="Button 1")
button_2 = Button(label="Button 2")
 
# put the results in a row
show(column(button_1, slider, button_group, select, button_2, width=300))
# show(row(button_1, slider, button_group, select, button_2))
# show(gridplot([[button_1, slider], [button_group, select],[button_2,]]))